In [3]:
# import seaborn as sns
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pickle
from gaf import GramianAngularField 
from pathlib import Path

In [5]:
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy) + 1])
    yy1[np.arange(len(yy)), yy] = 1
    return yy1


def generate_gaf(sin_data, image_size):
    min_ = np.amin(sin_data)
    max_ = np.amax(sin_data)
    scaled_serie = (2 * sin_data - max_ - min_) / (max_ - min_)
    gasf = GramianAngularField(image_size=image_size, method='summation',) #sample_range=(min_, max_)
    sin_gasf = gasf.transform(scaled_serie)
    gadf = GramianAngularField(image_size=image_size, method='difference',)
    sin_gadf = gadf.transform(scaled_serie)
    images = [sin_gasf[0], sin_gadf[0]]
    return images

In [5]:
np.random.seed(42)
mods = ['BPSK']
snrs = list(range(-20, 21, 1))

current_dir = Path('img_gaf.ipynb').parent

base_dir = current_dir / 'DATASET_FOR_GAF_1_128'

In [10]:


x_train = np.load(base_dir / "x_train.npy", allow_pickle=True)
x_test = np.load(base_dir / "x_test.npy", allow_pickle=True)
x_val = np.load(base_dir / "x_val.npy", allow_pickle=True)

w, h, c = 128, 128, 3

x_train_imgs = []
x_test_imgs = []
x_val_imgs = []

# x_train에 대한 GAF 이미지 생성
for i in tqdm(range(x_train.shape[0])):
    data = x_train[i, 0, :]  # 길이 128의 1차원 배열로 데이터 접근
    data = data.reshape(1, -1)  # (1, 128)로 reshape
    gaf_images = generate_gaf(data, 128)  # GAF 이미지 생성
    resultant = cv2.resize(gaf_images[0], (w, h))  # 원하는 크기로 리사이즈
    x_train_imgs.append(resultant)

# x_train GASF 이미지 저장
np.save(base_dir / "x_train_gasf.npy", x_train_imgs)

# x_test에 대한 GAF 이미지 생성
for i in tqdm(range(x_test.shape[0])):
    data = x_test[i, 0, :]  # 길이 128의 1차원 배열로 데이터 접근
    data = data.reshape(1, -1)  # (1, 128)로 reshape
    gaf_images = generate_gaf(data, 128)  # GAF 이미지 생성
    resultant = cv2.resize(gaf_images[0], (w, h))  # 원하는 크기로 리사이즈
    x_test_imgs.append(resultant)

# x_test GASF 이미지 저장
np.save(base_dir / "x_test_gasf.npy", x_test_imgs)

# x_val에 대한 GAF 이미지 생성
for i in tqdm(range(x_val.shape[0])):
    data = x_val[i, 0, :]  # 길이 128의 1차원 배열로 데이터 접근
    data = data.reshape(1, -1)  # (1, 128)로 reshape
    gaf_images = generate_gaf(data, 128)  # GAF 이미지 생성
    resultant = cv2.resize(gaf_images[0], (w, h))  # 원하는 크기로 리사이즈
    x_val_imgs.append(resultant)

# x_val GASF 이미지 저장
np.save(base_dir / "x_val_gasf.npy", x_val_imgs)

print("GASF 이미지 생성 및 저장 완료")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


100%|██████████| 12300/12300 [00:08<00:00, 1439.87it/s]


GASF 이미지 생성 및 저장 완료


# 채널 분리

In [6]:
train = np.load(base_dir / "x_train_gasf.npy")
print(train.shape)
print(train)

(98400, 128, 128)
[[[ 1.  1. -1. ...  1. -1. -1.]
  [ 1.  1. -1. ...  1. -1. -1.]
  [-1. -1.  1. ... -1.  1.  1.]
  ...
  [ 1.  1. -1. ...  1. -1. -1.]
  [-1. -1.  1. ... -1.  1.  1.]
  [-1. -1.  1. ... -1.  1.  1.]]

 [[ 1.  1. -1. ...  1.  1.  1.]
  [ 1.  1. -1. ...  1.  1.  1.]
  [-1. -1.  1. ... -1. -1. -1.]
  ...
  [ 1.  1. -1. ...  1.  1.  1.]
  [ 1.  1. -1. ...  1.  1.  1.]
  [ 1.  1. -1. ...  1.  1.  1.]]

 [[ 1.  1. -1. ... -1.  1. -1.]
  [ 1.  1. -1. ... -1.  1. -1.]
  [-1. -1.  1. ...  1. -1.  1.]
  ...
  [-1. -1.  1. ...  1. -1.  1.]
  [ 1.  1. -1. ... -1.  1. -1.]
  [-1. -1.  1. ...  1. -1.  1.]]

 ...

 [[ 1.  1. -1. ...  1. -1. -1.]
  [ 1.  1. -1. ...  1. -1. -1.]
  [-1. -1.  1. ... -1.  1.  1.]
  ...
  [ 1.  1. -1. ...  1. -1. -1.]
  [-1. -1.  1. ... -1.  1.  1.]
  [-1. -1.  1. ... -1.  1.  1.]]

 [[ 1.  1.  1. ...  1. -1. -1.]
  [ 1.  1.  1. ...  1. -1. -1.]
  [ 1.  1.  1. ...  1. -1. -1.]
  ...
  [ 1.  1.  1. ...  1. -1. -1.]
  [-1. -1. -1. ... -1.  1.  1.]
  [-1. -1.